# Basic Analysis
Slicing, Dicing, and Graphing.  Simple exploration of the data.

In [7]:
import pandas as pd
data = pd.read_csv('../raw_data/roxy_log.201607211200.csv', encoding='iso-8859-1')
col_map = {
    'Time (sec)': 'time',
    'A/F Sens 1 Ratio (AFR)': 'afr',
    'Boost (PSI)': 'boost',
    'Calculated Load (g/rev)': 'load',
    'Feedback Knock (°)': 'feedback_knock',
    'Fine Knock Learn (°)': 'fine_knock_learn',
    'Gear Position (Gear)': 'gear_position',
    'Ignition Timing (°)': 'timing',
    'MAF (g/s)': 'maf',
    'Primary Ign. (°)': 'primary_timing',
    'RPM (RPM)': 'rpm',
    'TD Burst (%)': 'td_burst',
    'TD Continuous (%)': 'td_continuous',
    'Target Boost (PSI)': 'boost_target',
    'Throttle Pos. (%)': 'throttle',
    'Wastegate Duty (%)': 'wg_duty',
    'Wastegate Max (%)': 'wg_max',
}
data.rename(columns=col_map, inplace=True)

# fix timestamps
from os import listdir
from os.path import isfile, join

raw_data_path = '../raw_data/'
logfiles = [f for f in listdir(raw_data_path) if isfile(join(raw_data_path, f))]
start_times = [pd.Timestamp(log.split('.')[1]) for log in logfiles]

delta_t = pd.to_timedelta(data['time'], unit='s')
data['time'] = delta_t + start_times[0]
# set the series as the new index
data.index = data.time
# and drop the duplicate column
data = data.drop('time', 1)

In [8]:
# describe some high-level stats about the data
data.describe()

afr         boost          load  feedback_knock  \
count  19756.000000  19756.000000  19756.000000    19756.000000   
mean      15.538034     -7.721658      0.443494       -0.000854   
std        3.233380      4.825361      0.406604        0.042452   
min       11.250000    -12.300000      0.030000       -2.110000   
25%       14.360000     -9.980000      0.270000        0.000000   
50%       14.590000     -9.130000      0.310000        0.000000   
75%       14.700000     -7.660000      0.460000        0.000000   
max       25.610000     18.600000      2.740000        0.000000   

       fine_knock_learn  gear_position        timing           maf  \
count      19756.000000   19756.000000  19756.000000  19756.000000   
mean          -0.042994       3.521715     22.365079     18.249331   
std            0.241604       2.106500     11.021122     30.369333   
min           -2.110000       1.000000    -11.000000      1.560000   
25%            0.000000       1.000000     11.500000      3.920000   
50%            0.000000       4.000000     20.000000      5.470000   
75%            0.000000       6.000000     34.000000     19.430000   
max            0.000000       6.000000     42.500000    231.480000   

       primary_timing           rpm      td_burst  td_continuous  \
count    19756.000000  19756.000000  19756.000000   19756.000000   
mean        22.643487   1971.443106     -0.003020       4.361465   
std         11.718651   1196.140508      0.182049       1.469534   
min          5.310000    667.000000     -5.030000      -5.030000   
25%         11.990000    733.000000      0.000000       5.000000   
50%         16.910000   1737.500000      0.000000       5.000000   
75%         35.550000   3017.000000      0.000000       5.000000   
max         42.580000   6880.000000      4.810000       5.000000   

       boost_target      throttle       wg_duty        wg_max  \
count  19756.000000  19756.000000  19756.000000  19756.000000   
mean      -4.127713      7.507795      8.866477     12.057363   
std        4.318753     16.144262     14.149493     15.502971   
min       -8.160000      0.000000      0.000000      0.000000   
25%       -5.340000      0.000000      0.000000      0.000000   
50%       -5.340000      1.000000      0.000000      0.000000   
75%       -5.340000     10.000000     21.180000     31.660000   
max       17.980000    100.000000     45.490000     45.660000   

       AP Info:[AP3-SUB-001 v1.7.2.0-11034][USDM 2004 Impreza WRX][Reflash: EXPERIMENTAL-TP-cobb2.ptm - Realtime: EXPERIMENTAL-TP-cobb2.ptm]  
count                                            19756.0                                                                                      
mean                                                 0.0                                                                                      
std                                                  0.0                                                                                      
min                                                  0.0                                                                                      
25%                                                  0.0                                                                                      
50%                                                  0.0                                                                                      
75%                                                  0.0                                                                                      
max                                                  0.0

In [14]:
# slice out just the boost column
boost = data['boost']
print( boost.head() )
print( boost.describe() )

time
2016-07-21 08:00:00.000   -7.19
2016-07-21 08:00:00.004   -7.19
2016-07-21 08:00:00.014   -7.19
2016-07-21 08:00:00.016   -7.14
2016-07-21 08:00:00.019   -7.08
Name: boost, dtype: float64
count    19756.000000
mean        -7.721658
std          4.825361
min        -12.300000
25%         -9.980000
50%         -9.130000
75%         -7.660000
max         18.600000
Name: boost, dtype: float64


In [16]:
# get just boost above 10psi
boost[boost > 10].count()

435

In [17]:
# get just boost above 10psi
boost[boost > 10].count()


435

In [19]:
# look for learned knock events
knock = data['fine_knock_learn']
data[knock < 0].describe()

afr       boost        load  feedback_knock  fine_knock_learn  \
count  665.000000  665.000000  665.000000      665.000000        665.000000   
mean    12.631429    6.940496    1.685203       -0.015865         -1.277263   
std      1.686369    7.423979    0.646754        0.182408          0.397252   
min     11.250000   -3.690000    0.900000       -2.110000         -2.110000   
25%     11.250000   -0.230000    1.050000        0.000000         -1.760000   
50%     11.370000    5.220000    1.510000        0.000000         -1.050000   
75%     14.130000   14.480000    2.360000        0.000000         -1.050000   
max     22.390000   18.600000    2.740000        0.000000         -0.700000   

       gear_position      timing         maf  primary_timing          rpm  \
count     665.000000  665.000000  665.000000      665.000000   665.000000   
mean        3.150376   19.912030  112.654015       17.947038  3658.215038   
std         1.522404    7.682626   68.340908        9.967908  1212.798956   
min         1.000000    5.500000   28.900000        6.020000  1751.000000   
25%         2.000000   13.000000   42.350000        9.880000  2334.000000   
50%         3.000000   18.000000  106.220000       14.100000  3996.000000   
75%         4.000000   27.000000  193.090000       27.110000  4515.000000   
max         6.000000   40.000000  231.480000       40.470000  6769.000000   

         td_burst  td_continuous  boost_target    throttle     wg_duty  \
count  665.000000     665.000000    665.000000  665.000000  665.000000   
mean    -0.028767       2.221699      8.563068   53.234586   28.900571   
std      0.344551       2.216265      7.430908   35.475840   10.378881   
min     -5.030000      -5.030000     -7.930000    0.000000    0.000000   
25%      0.000000       0.660000      1.990000   24.000000   22.750000   
50%      0.000000       1.970000      6.880000   35.000000   29.020000   
75%      0.000000       5.000000     17.850000  100.000000   36.080000   
max      1.230000       5.000000     17.980000  100.000000   45.490000   

           wg_max  \
count  665.000000   
mean    30.191579   
std      9.814778   
min      0.630000   
25%     24.760000   
50%     30.210000   
75%     37.340000   
max     45.660000   

       AP Info:[AP3-SUB-001 v1.7.2.0-11034][USDM 2004 Impreza WRX][Reflash: EXPERIMENTAL-TP-cobb2.ptm - Realtime: EXPERIMENTAL-TP-cobb2.ptm]  
count                                              665.0                                                                                      
mean                                                 0.0                                                                                      
std                                                  0.0                                                                                      
min                                                  0.0                                                                                      
25%                                                  0.0                                                                                      
50%                                                  0.0                                                                                      
75%                                                  0.0                                                                                      
max                                                  0.0

### Filter down to just OverBoost Events

In [22]:
# return a filterd dataframe, containing only samples where boost > boost_target
overboost = data[data.boost > data.boost_target]
overboost.describe()

afr       boost        load  feedback_knock  fine_knock_learn  \
count  604.000000  604.000000  604.000000      604.000000        604.000000   
mean    12.974702    1.868626    1.225447       -0.013974         -0.274189   
std      1.688190    7.515390    0.649270        0.171282          0.561090   
min     11.250000   -7.870000    0.140000       -2.110000         -2.110000   
25%     11.250000   -4.000000    0.730000        0.000000          0.000000   
50%     12.980000   -1.760000    0.910000        0.000000          0.000000   
75%     14.360000    7.537500    1.695000        0.000000          0.000000   
max     23.770000   18.600000    2.740000        0.000000          0.000000   

       gear_position      timing         maf  primary_timing          rpm  \
count     604.000000  604.000000  604.000000      604.000000   604.000000   
mean        3.307947   21.403146   66.278526       23.758427  2989.761589   
std         1.922513    9.971941   52.574457       11.827228  1195.899714   
min         1.000000    4.500000   10.000000        5.660000   680.000000   
25%         1.000000   13.500000   21.962500       11.640000  1720.000000   
50%         3.000000   19.750000   46.095000       24.825000  3370.000000   
75%         5.000000   30.000000  104.677500       34.140000  3846.500000   
max         6.000000   41.500000  216.750000       42.580000  6769.000000   

         td_burst  td_continuous  boost_target    throttle     wg_duty  \
count  604.000000     604.000000    604.000000  604.000000  604.000000   
mean    -0.229123      -1.262219      0.541175   23.794702   16.513344   
std      0.871073       1.669072      7.681104   24.905145   15.767312   
min     -5.030000      -5.030000     -8.160000    0.000000    0.000000   
25%      0.000000      -1.975000     -5.320000   10.000000    0.000000   
50%      0.000000      -0.750000     -3.270000   14.000000   15.885000   
75%      0.000000       0.000000      6.170000   28.000000   30.980000   
max      1.360000       5.000000     17.980000  100.000000   45.490000   

           wg_max  \
count  604.000000   
mean    23.604354   
std     16.665541   
min      0.000000   
25%      2.260000   
50%     33.465000   
75%     35.430000   
max     45.660000   

       AP Info:[AP3-SUB-001 v1.7.2.0-11034][USDM 2004 Impreza WRX][Reflash: EXPERIMENTAL-TP-cobb2.ptm - Realtime: EXPERIMENTAL-TP-cobb2.ptm]  
count                                              604.0                                                                                      
mean                                                 0.0                                                                                      
std                                                  0.0                                                                                      
min                                                  0.0                                                                                      
25%                                                  0.0                                                                                      
50%                                                  0.0                                                                                      
75%                                                  0.0                                                                                      
max                                                  0.0

In [25]:
# get just the boost value, where boost > target
data[data.boost > data.boost_target]['boost'].describe()
# Best practice is to use loc method:
data.loc[data.boost > data.boost_target, 'boost'].describe()

count    604.000000
mean       1.868626
std        7.515390
min       -7.870000
25%       -4.000000
50%       -1.760000
75%        7.537500
max       18.600000
Name: boost, dtype: float64

NOTE: looks like almost all of our overboost events are where boost is negative.

Probably should adjust the map so that we target more reasonable boost levels for these areas.

consider setting minimum boost targets to 0psi, and adjusting the max WGDC so that it is the MINIMUM WGDC needed to hit peak boost.
Or, perhaps, because the WG will only be able to open above ~7psi [WG rod begins to aggressively move at ~7psi](https://cobbtuning.zendesk.com/hc/en-us/articles/200025164-How-Subaru-s-Factory-Boost-Control-System-Works), set the minimum targets to that value, WGDC will be irrelevant below the 7psi threshold.

In [26]:
# filter rows based on multiple criteria:
# use &, |, and parens for evaluation order.
data[(data.boost > 0) & (data.boost > data.boost_target)].describe()

afr       boost        load  feedback_knock  fine_knock_learn  \
count  264.000000  264.000000  264.000000      264.000000        264.000000   
mean    11.831932    9.097689    1.838561       -0.031970         -0.583561   
std      1.109689    5.720240    0.518190        0.258237          0.711704   
min     11.250000    0.020000    0.480000       -2.110000         -2.110000   
25%     11.250000    4.347500    1.410000        0.000000         -1.050000   
50%     11.250000    9.220000    1.830000        0.000000          0.000000   
75%     11.830000   13.150000    2.155000        0.000000          0.000000   
max     15.730000   18.600000    2.740000        0.000000          0.000000   

       gear_position      timing         maf  primary_timing          rpm  \
count     264.000000  264.000000  264.000000      264.000000   264.000000   
mean        4.488636   16.899621  115.020909       13.074356  3729.901515   
std         1.275636    4.958992   42.831166        6.948428   561.283332   
min         1.000000   10.000000   43.840000        5.660000  2324.000000   
25%         3.000000   13.375000   83.285000        8.120000  3488.250000   
50%         4.000000   15.500000  109.120000       10.590000  3737.500000   
75%         6.000000   19.000000  130.507500       14.890000  4031.750000   
max         6.000000   33.000000  216.750000       42.580000  5720.000000   

         td_burst  td_continuous  boost_target    throttle     wg_duty  \
count  264.000000     264.000000    264.000000  264.000000  264.000000   
mean    -0.212652      -1.344167      7.631515   40.678030   29.364242   
std      0.846607       1.808022      6.490554   28.799796   10.486559   
min     -5.030000      -5.030000     -8.100000    0.000000    0.000000   
25%      0.000000      -1.997500      2.565000   21.000000   23.042500   
50%      0.000000      -0.500000      8.480000   32.000000   30.590000   
75%      0.000000       0.000000     11.547500   42.250000   35.690000   
max      0.000000       5.000000     17.980000  100.000000   45.490000   

           wg_max  \
count  264.000000   
mean    36.224962   
std      5.630104   
min      7.410000   
25%     35.160000   
50%     35.935000   
75%     37.370000   
max     45.660000   

       AP Info:[AP3-SUB-001 v1.7.2.0-11034][USDM 2004 Impreza WRX][Reflash: EXPERIMENTAL-TP-cobb2.ptm - Realtime: EXPERIMENTAL-TP-cobb2.ptm]  
count                                              264.0                                                                                      
mean                                                 0.0                                                                                      
std                                                  0.0                                                                                      
min                                                  0.0                                                                                      
25%                                                  0.0                                                                                      
50%                                                  0.0                                                                                      
75%                                                  0.0                                                                                      
max                                                  0.0